# Quote Generation/Retrieval

Initial quotes were scraped from one webpage. But that would lead to quite a lot of repitition and kindof a boring bot. So...

### 2 Issues with Index retrieved/Generated Quotes:
**Goal:** To have ~100 quotes to have tweeted on a rolling basis since this is a quarter of a year and will not likely to be easily repeated.

### Probability of Repeating a Quote Exactly 2 Weeks Later

\begin{equation*}
P(\text{repeated in 2 weeks}) = 1 - \left( \frac{N - 1}{N} \right)^{\frac{14}{m}}
\end{equation*}

 Substituting in the values:

 \begin{equation*}
P(\text{repeated in 2 weeks}) = 1 - \left( \frac{99}{100} \right)^{14}
\end{equation*}

 Calculating this gives:

 \begin{equation*}
P(\text{repeated in 2 weeks}) \approx 0.135
\end{equation*}

Personally, I'd like it to be under 10% for a longer period of time (increases probability)

Tweaking the numbers a bit 
N = 200
days = 21

\begin{equation*}
P(\text{repeated in 3 weeks}) \approx  0.0980
\end{equation*}

The following should be conditional 1) That the quote generated has confidence that the quote was written by Ovid then 2) That the quote is not a variation of a quote already in DB.

In [10]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
import os
import openai
import pandas as pd

from itertools import product

# import utils
import configparser
from pprint import pprint
from llama_index import SimpleDirectoryReader


import os
import openai


import jupyter_black

jupyter_black.load()
# Access values from the sections

import sys

sys.path.append("../app")
import conn_utils

OPENAI_API_KEY = conn_utils.get_open_ai_key("./../app/config.ini")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
directory = "index_store"

pd.set_option("display.max_colwidth", None)

In [2]:
!ls ./../data

Amours.txt                 MetamorphosesI_VII.txt
Fasti.txt                  MetamorphosesVIII_XV.txt
Heroides.txt               MetamorphosesofPublius.txt
LastPoems.txt              RemediaAmoris.txt
LoversAssistant.txt


## Get low sentency similarity

Instead of using a LLM. You can trust to get a close (but not too close) similarity

In [6]:
import spacy

# Read in exisitng quotes
df = pd.read_json("./../app/ovid_quotes.json")

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Define the file paths
files = [
    "RemediaAmoris.txt",
    "Heroides.txt",
    "Amours.txt",
    "Fasti.txt",
    "MetamorphosesI_VII.txt",
    "MetamorphosesVIII_XV.txt",
    "MetamorphosesofPublius.txt",
    "LoversAssistant.txt",
    "LastPoems.txt",
]

# Read in the text files and split sentences
from tqdm import tqdm

sentences = []
for file in tqdm(["MetamorphosesI_VII.txt", "MetamorphosesVIII_XV.txt"]):
    with open(os.path.join("./../data", file), "r") as f:
        text = f.read()
        doc = nlp(text)
        sentences.extend([sent.text for sent in doc.sents])

# Print the list of sentences
print(sentences)

100%|██████████| 2/2 [00:55<00:00, 27.86s/it]

['\ufeffThe Project Gutenberg eBook of The Metamorphoses of Ovid, Books I-VII\n    \nThis ebook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever.', 'You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this ebook or online\nat www.gutenberg.org.', 'If you are not located in the United States,\nyou will have to check the laws of the country where you are located\nbefore using this eBook.\n\n', 'Title: The Metamorphoses of Ovid, Books I-VII\n\n\nAuthor: Ovid\n\nTranslator: Henry T. Riley\n\nRelease date: June 8, 2007 [eBook #21765]\n                Most recently updated: June 28, 2021\n\nLanguage: English\n\n\n\n*** START OF THE PROJECT GUTENBERG EBOOK THE METAMORPHOSES OF OVID, BOOKS I-VII ***\n\n\n\n  ', '[Transcriber’s Note:\n\n  In the original text, words and phrases supplied by the translator\n  were printed in _italics_.', 'In this e

In [7]:
t = product(sentences, df["Quote"])
t = list(t)

In [8]:
similarity_data = []

for generated_quote, actual_quote in tqdm(t):
    # Process the quotes with spaCy
    doc_generated = nlp(generated_quote)
    doc_actual = nlp(actual_quote)

    # Calculate similarity score
    similarity_score = doc_generated.similarity(doc_actual)

    # Append data to the list
    similarity_data.append(
        {
            "new_quotes": generated_quote,
            "actual_quotes": actual_quote,
            "similarity_score": similarity_score,
        }
    )

# Create a DataFrame from the similarity data
similarity_df = pd.DataFrame(similarity_data).sort_values(
    "similarity_score", ascending=False
)

  0%|          | 0/411872 [00:00<?, ?it/s]/var/folders/qq/gdppvj912kv3ds7_xnf0q_fc0000gn/T/ipykernel_21764/1207181298.py:9: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_score = doc_generated.similarity(doc_actual)
100%|██████████| 411872/411872 [1:02:23<00:00, 110.02it/s]


In [28]:
upper_similarity = 0.8
lower_similarity = 0.2

suitable_df = similarity_df[
    (similarity_df["similarity_score"] > lower_similarity)
    & (similarity_df["similarity_score"] < upper_similarity)
].sort_values(
    "similarity_score", ascending=True
)  # .sample(10)
print(suitable_df.shape)
# Clean out very short quotes
suitable_df = suitable_df[suitable_df["new_quotes"].str.len() > 15]
suitable_df = suitable_df[~suitable_df["new_quotes"].str.contains("Footnote")]
print(suitable_df.shape)
suitable_df.sample(10)

(293773, 3)
(244619, 3)


,new_quotes,actual_quotes,similarity_score
37502,"And that heaven\nand earth may endure equal heat, neither drive too low, nor urge the\nchariot along the summit of the sky.",The mind moves in the direction of our currently dominant thoughts.,0.400273
349416,"His woolly flocks\nfollowed, there being no one to guide them.",There is a god within us.,0.390001
113143,"Clarke\n translates this line, ‘He avoids the monster’s eager snaps with\n his swift wings.’]\n\n","Everything changes, nothing perishes.",0.347971
302235,"The sea is raging in a hurricane[43] so vast, and all\nthe sky is concealed beneath the shade brought on by the clouds of\npitchy darkness, and the face of the night is redoubled {in gloom}.","Dripping water hollows out stone, not through force but through persistence.",0.381407
175570,"This, they suppose, grew solid, and, receiving the\nnourishment of a fruitful and productive soil, acquired the power of\nbeing noxious.","Everything changes, nothing perishes.",0.206893
388732,"Pliny the\n Elder, in his thirty-second Book, says-- ‘Theophrastus tells us\n that Crathis, a river of the Thurians, produces whiteness, whereas\n the Sybaris causes blackness, in sheep and cattle.","Happy is the man who has broken the chains which hurt the mind, and has given up worrying once and for all.",0.295774
24038,"Such a burden as that\n{better} befits my shoulders; I, who am able to give unerring wounds to\nthe wild beasts, {wounds} to the enemy, who lately slew with arrows\ninnumerable the swelling Python, that covered so many acres {of land}\nwith his pestilential belly.","Love yields to business. If you seek a way out of love, be busy; you'll be safe then.",0.357960
171167,Lofty habitation._--Ver.,Nothing is stronger than habit.,0.358452
158552,"This was the fleece of the\n ram that carried Phryxus along the Hellespont to Colchis, which is\n mentioned again in the next Book.]\n\n",Fortune and love favor the brave.,0.250310
75764,And\nshe confirmed her threats by the event.,The cause is hidden. The effect is visible to all.,0.243098


In [27]:
## Update add new quotes to the dataframe
# new_row = {
#     "Quote": "And did you, my hands, seize the horns of the mighty bull?",
#     "Work": "Metamorphoses",
#     "Quote in Latin": "Tenuistine, manus meae, cornua tauri magni?",
# }
# df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
# df.to_json("./../app/ovid_quotes.json")

In [ ]:
df = pd.read_json("./../app/ovid_quotes.json")

DATA_DIR = "./../data"
files = [
    "RemediaAmoris.txt",
    "Heroides.txt",
    "Amours.txt",
    "Fasti.txt",
    "MetamorphosesI_VII.txt",
    "MetamorphosesVIII_XV.txt",
    "MetamorphosesofPublius.txt",
    "LoversAssistant.txt",
    "LastPoems.txt",
]
docs = [f"{DATA_DIR}/{file}" for file in files]
documents = SimpleDirectoryReader(input_files=docs).load_data()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=25)
texts = text_splitter.create_documents(docs)
directory = "index_store"
vector_index = FAISS.from_documents(texts, OpenAIEmbeddings())
vector_index.save_local(directory)

vector_index = FAISS.load_local("index_store", OpenAIEmbeddings())
retriever = vector_index.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [ ]:
df = pd.read_json("./../app/ovid_quotes.json")
df.head(1)

In [ ]:
qa_interface = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

response = qa_interface(
    f"""
I am a big fan of ovid. 
Please recommend 10 memorable quotes to me along with the source document they were taken from.

Do NOT include quotes I already have:
{df["Quote"].tolist()}
"""
)

print(response["result"])

#### Authorship authentication

**[Who Wrote it and Why?
Prompting Large-Language Models for Authorship Verification](https://arxiv.org/pdf/2310.08123.pdf)**

In [ ]:
# LLM Authorship Attribution
from typing import List

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser


class CommaSeparatedListOutputParser(BaseOutputParser[List[str]]):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        return text.strip().split(", ")


template = f"""Task: On a scale of 0 to 1, with 0 indicating low confidence
and 1 indicating high confidence, please provide a general
assessment of the likelihood that given text 
written by the same author as the provided reference. Your answer should reflect a
moderate level of strictness in scoring. Here are some
relevant variables to this problem.
1. punctuation style(e.g. hyphen, brackets, colon, comma,
parenthesis, quotation mark)
2. special characters style, capitalization style(e.g.
Continuous capitalization, capitalizing certain words)
3. acronyms and abbreviations(e.g. Usage of acronyms
such as OMG, Abbreviations without punctuation marks
such as Mr Rochester vs. Mr. Rochester,Unusual
abbreviations such as def vs. definitely)
4. writing style
5. expressions and Idioms
6. tone and mood
7. sentence structure
8. any other relevant aspect
First step: Understand the problem, extracting relevant
variables and devise a plan to solve the problem. Then,
carry out the plan and solve the problem step by step.
9. One (or both) of the texts is written by the famous Latin author "Ovid"
Finally, show the confidence score.

The following are all quotes by Ovid for reference:
'Love is a thing full of anxious fears.',
 'Now are fields of corn where Troy once stood.',
 "We're slow to believe what wounds us.",
 "The end proves the acts (were done), or the result is a test of the actions; Ovid's line 85 full translation: “The event proves well the wisdom of her [Phyllis'] course.”",
 "Let him who loves, where love success may find, Spread all his sails before the prosp'rous wind.",
 'Resist beginnings; the remedy comes too late when the disease has gained strength by long delays.',
 "Love yields to business. If you seek a way out of love, be busy; you'll be safe then.",
 'The gods behold all righteous actions.',
 'There is a god within us.',
 'The mind, conscious of rectitude, laughed to scorn the falsehood of report.',
 'Every lover is a soldier.',
 'Let the man who does not wish to be idle fall in love!',
 'Far away be that fate!',
 'They bear punishment with equanimity who have earned it.',
 "We take no pleasure in permitted joys. But what's forbidden is more keenly sought.",
 'Who is allowed to sin, sins less.' 
 
 """
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("human", human_template),
    ]
)
chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()

In [ ]:
# Extract Quotes from LLM Authorshipp Attribution
import pandas as pd
import spacy
import re

nlp = spacy.load("en_core_web_sm")
input_string = response["result"]


def get_quotes(input_string):
    """Extract quotes from a string that has been created by an LLM string prompt."""
    # Extract the lines that start with a number
    lines_with_stripped_numbers = [
        re.sub(r"^\d+\.\s*", "", line.strip())
        for line in input_string.splitlines()
        if re.match(r"^\d+\.", line)
    ]

    # Print the extracted lines
    quotes = []
    pattern2 = r'"([^"]+)"\s*-\s*(.+)'

    for line in lines_with_stripped_numbers:
        match = re.search(pattern2, line)
        q = match.group(1)
        w = match.group(2)
        quotes.append((q, w))
    return quotes


quotes = get_quotes(response["result"])
quotes

### Score New Quotes

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


def score_quotes(quotes=None):
    """Score a list of quotes that have been generated by an LLM string prompt.

    quotes example:
    [('Dripping water hollows out stone, not through force but through persistence.',
    'Metamorphoses'),

    """
    if quotes is None:
        quotes = []

    # Define the regex pattern
    pattern = r"(\d+\.\d+)"

    # Get the quote string (not the work)
    quotes_text = [q[0] for q in quotes]
    quotes_work = [q[1] for q in quotes]

    # Extract the scores
    scores = []
    score_reasons = []
    for q in quotes_text:
        r = chain.invoke({"text": q})
        r = " ".join(r)
        score_reasons.append(r)
        match = re.search(pattern, r)
        score = match[0] if match else -1
        scores.append(score)
    return list(zip(*(scores, score_reasons, quotes_text, quotes_work)))


# Example usage
scored_quotes = score_quotes(quotes)
scored_quotes_df = pd.DataFrame(
    scored_quotes, columns=["score", "reason", "quote", "work"]
)
pd.set_option('display.max_colwidth', None)
scored_quotes_df  # ["score"]

In [ ]:
scored_quotes_df["quote"].tolist()

### Context-Based Quotation Recommendation

Resource:
* https://arxiv.org/pdf/2005.08319.pdf

### Assessment of Retrieval Accuracy

The below uses current pipeline for 2 reasons:
1) To assess the hulicination affect against current pipeline
2) To assess scoring variability

In [ ]:
import re

non_ovid_generated_quotes = [
    "One man's meat is another man's poison.",
    "Fortune favors the bold.",
    "Wherever there is a human being, there is an opportunity for a kindness.",
    "Love is a kind of warfare.",
    "One man's meat is another man's poison.",
    "To be loved, be lovable.",
]


# Define the regex pattern
pattern = r"(\d+\.\d+)"


scores = []
strings = []
for q in non_ovid_generated_quotes:
    r = chain.invoke({"text": q})
    r = " ".join(r)
    strings.append(r)
    match = re.search(pattern, r)
    score = match[0] if match else -1
    scores.append(score)
    print(r)

pd.DataFrame(
    data={
        "questionable_quote": non_ovid_generated_quotes,
        "authorship_match_score": scores,
        "is_original": [0, 0, 0, 1, 0, 1],
    }
)

In [ ]:
scored_quotes_df[scored_quotes_df["score"].astype(float) < 0.5]["quote"].tolist()

In [ ]:
from itertools import product

# Compare to quotes
generated_quotes = scored_quotes_df[scored_quotes_df["score"].astype(float) > 0.5][
    "quote"
].tolist()

# Read in existing quotes
df = pd.read_json("./../app/ovid_quotes.json")

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Process each quote and calculate similarity scores
similarity_data = []

for generated_quote, actual_quote in product(generated_quotes, df["Quote"]):
    # Process the quotes with spaCy
    doc_generated = nlp(generated_quote)
    doc_actual = nlp(actual_quote)

    # Calculate similarity score
    similarity_score = doc_generated.similarity(doc_actual)

    # Append data to the list
    similarity_data.append(
        {
            "generated_quotes": generated_quote,
            "actual_quotes": actual_quote,
            "similarity_score": similarity_score,
        }
    )

# Create a DataFrame from the similarity data
similarity_df = pd.DataFrame(similarity_data).sort_values(
    "similarity_score", ascending=False
)
display(similarity_df.head(2))

# Seems like there might be a magic number similarity score < 0.65
print("unique quutes")

similiar_quotes = similarity_df[similarity_df["similarity_score"] > 0.65][
    "generated_quotes"
].unique()

# Filter out the similar quotes
new_quotes = similarity_df[~similarity_df["generated_quotes"].isin(similiar_quotes)][
    "generated_quotes"
].unique()

# Get the works for the new quotes
new_quote_works = []
for q in new_quotes:
    for quote, work in quotes:
        if q == quote:
            new_quote_works.append(work)
df_new_quotes = pd.DataFrame(
    list(zip(*(new_quotes, new_quote_works))), columns=["Quote", "Work"]
)
df_new_quotes["Quote in Latin"] = None
df_new_quotes

In [ ]:
similiar_quotes

In [ ]:
print("highest score comparison:")
idx = 54
print("GENERATED:", similarity_df.iloc[54]["generated_quotes"])
print("ACTUAL:", similarity_df.iloc[54]["actual_quotes"])

### Storage of Additional Quotes

In [ ]:
!ls ./../

In [ ]:
# After quotes works have passed the assessment bars above
df = pd.concat([df, df_new_quotes]).reset_index(drop=True)
df.to_json("./../ovid_quotes.json")

## Fine-Tuning

**Format:** Example data from OpenAI for .jsonl
```jsonl
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, 
            {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", 
            "content": "Paris, as if everyone doesn't know that already."}]}
```